In [1]:
Libdl.find_library(["libDsp"])

"libDsp"

In [2]:
using JuMP, Dsp, Distributions

In [30]:
srand(123)

CROPS  = 1:3 # set of crops (wheat, corn and sugar beets, resp.)
PURCH  = 1:2 # set of crops to purchase (wheat and corn, resp.)
SELL   = 1:4 # set of crops to sell (wheat, corn, sugar beets under 6K and those over 6K)

NS = 100;                    # number of scenarios
NP = 3;
S = collect(1:NS)           # scenario set
P = collect(1:NP)           # set of crops (1=wheat,2=corn,3=beets)

probability = ones(NS)/NS

Cost     = [150 230 260]   # cost of planting crops
Budget   = 500             # budget capacity
Purchase = [238 210];      # purchase price
Sell     = [170 150 36 10] # selling price
Minreq   = [200 240 0]     # minimum crop requirement


# assign random data
Yield = zeros(length(S),NP)
d = Uniform(5,20)
for j in 1:(NP)
    Yield[S,j] = rand(d,NS)[S]
end
#d = Uniform(10,30)
#Yield[S,NP] = rand(d,NS)[S];

Budget = 500

# JuMP model
m = Model(NS)

@variable(m, x[i=CROPS] >= 0, Int)
@objective(m, Min, sum(Cost[i] * x[i] for i=CROPS))
@constraint(m, const_budget, sum(x[i] for i=CROPS) <= Budget)

for s in blockids()
    blk = Model(m, s, probability[s])

    @variable(blk, y[j=PURCH] >= 0)
    @variable(blk, w[k=SELL] >= 0)

    @objective(blk, Min, sum(Purchase[j] * y[j] for j=PURCH) - sum(Sell[k] * w[k] for k=SELL))

    @constraint(blk, const_minreq[j=PURCH], Yield[s,j] * x[j] + y[j] - w[j] >= Minreq[j])
    @constraint(blk, const_minreq_beets, Yield[s,3] * x[3] - w[3] - w[4] >= Minreq[3])
    @constraint(blk, const_aux, w[3] <= 6000)
end


In [31]:
print(m)

Min 150 x[1] + 230 x[2] + 260 x[3]
Subject to
 x[1] + x[2] + x[3] ≤ 500
 x[i] ≥ 0, integer, ∀ i ∈ {1,2,3}


In [32]:
# Dsp solve types
solve_types = [:Dual, :Benders, :Extensive]

# Default parameter file
myparam = joinpath(dirname(@__FILE__),"DSP/parameters/default.txt")

status = solve(m, solve_type = solve_types[2], param = myparam)

Finding a good lower bound using Dual Decomposition...
CPLEX is not available for QP solve.

DUAL DECOMPOSITION ITERATION INFORMATION:
* master   = objective function value of the master problem.
* primobj  = best primal objective function value.
* dualobj  = best dual objective function value.
* a.gap(%) = Approximate gap between master and dualobj.
* d.gap(%) = Duality gap between primobj and dualobj.
* times    = wall clock time in seconds.

  iter         master        primobj        dualobj  a.gap(%)  d.gap(%)    time
 D   0  -1.035329e+06          Large  -1.069031e+06      3.26    100.00     0.2
 D   1  -9.679253e+05          Large  -1.040442e+06      7.49    100.00     0.7
 D   2  -9.005213e+05          Large  -1.005414e+06     11.65    100.00     1.1
 D   3  -8.331173e+05          Large  -1.004915e+06     20.62    100.00     1.4
     4  -9.560458e+05          Large  -1.004915e+06      5.11    100.00     1.8
 D   5  -9.307074e+05          Large  -9.599553e+05      3.14    100.00

:GapTolerance

In [33]:
getvalue(x)

x: 1 dimensions:
[1] = 485.0
[2] = 15.0
[3] = 0.0